In [ ]:
from dask.distributed import LocalCluster, Client
from dask import delayed, compute

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

import pandas as pd
import numpy as np

In [ ]:
cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
def make_rand(max_val: float=1e2, min_val: float=1e3, size: int=10) -> np.ndarray:
    arr = np.arange(1e2) + 1e3
    np.random.shuffle(arr)
    return arr[:10].astype(int)

In [ ]:
def make_data(size: float=1e3, subsize: int=10) -> pd.DataFrame:
    size = int(size)
    a, b = make_rand(size=subsize).astype(str), make_rand(size=subsize).astype(str)
    df = pd.DataFrame(dict(
        a=np.random.choice(a, size=size),
        b=np.random.choice(b, size=size),
        c=np.random.randint(1e3, size=size)
    ), columns=('a', 'b', 'c'))
    for label in ['a', 'b']:
        df.loc[:, label] = df.loc[:, label].astype('category')
    return df

In [ ]:
data = make_data(size=8e6, subsize=50)

In [ ]:
for key, group in data.groupby(['a', 'b']):
    print(key, len(group))

In [ ]:
data.groupby(['a', 'b']).count()

In [ ]:
df = dd.from_pandas(data, npartitions=cluster.n_workers)
ddata = client.persist(df)

In [ ]:
def func(df):
    return df.c.tolist()

In [ ]:
y = ddata.groupby(['a', 'b']).apply(func, meta=1)

In [ ]:
%%time
yy = y.compute()

In [ ]:
yy.head()

In [ ]:
a, b = yy.index[0]
msk = (ddata.a == a) & (ddata.b == b)
y = ddata.loc[msk]

In [ ]:
y.visualize()

In [ ]:
%%time
y.compute()

In [ ]:
def work(df):
    return df.c.shift(-1).dropna()

In [ ]:
for key, group in data.groupby(['a', 'b']):
    print(group.head())
    r = work(group)
    print(r.head())
    break

In [ ]:
y = ddata.groupby(['a', 'b']).apply(work, meta=1)

In [ ]:
y.visualize()

In [ ]:
%%time
yy = y.compute()

In [ ]:
yy